In [0]:
! pip install optuna

In [0]:
import numpy as np
import sklearn.metrics
from keras import layers
from keras import Input
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
import keras
from keras.wrappers.scikit_learn import KerasRegressor
import optuna
from sklearn.model_selection import train_test_split

In [0]:
'''
データのロード
'''
import pickle

pickle_file = "./tuned_selectedData.pickle"
with open(pickle_file, 'rb') as f:
    X_train = pickle.load(f)
    Y_train = pickle.load(f)
    X_test = pickle.load(f)

In [0]:
(X_train, X_val, Y_train, Y_val) = train_test_split(X_train, Y_train, test_size=0.2, random_state=666)

In [0]:
def create_model(n_layer, mid_units, optimizer, dropout_rate):
    model = Sequential()
    model.add(layers.Dense(mid_units, input_dim=len(X_train[0]), activation='relu'))
    for i in range(n_layer):
        model.add(layers.Dense(mid_units))
        model.add(BatchNormalization())
        model.add(layers.Activation('relu'))
        model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(1))

    return model

In [0]:
'''
ハイパーパラメータの最適化
'''
callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor="val_acc",
        patience=1
    )
]

def objective(trial):
    # hypyer param
    n_layer = trial.suggest_int('n_layer', 1, 12)
    mid_units = int(trial.suggest_discrete_uniform('mid_units', 2, 128, 2))
    optimizer = trial.suggest_categorical('optimizer', ['adam', 'rmsprop','adagrad','adadelta','nadam'])
    dropout_rate = trial.suggest_uniform('dropout_rate', 0, 1)
 
 
    # model
    model = create_model(
        n_layer, 
        mid_units,
        optimizer,
        dropout_rate
    )
    model.compile(optimizer=optimizer, loss="mse")
    
    # fit
    estimator_nn = KerasRegressor(build_fn = model, epochs = 20, batch_size = 16, verbose = 1)
    estimator_nn.fit(
              X_train, 
              Y_train, 
              eval_metric="rmse", 
              eval_set=[
                  (X_train, Y_train),
                  (X_val, Y_val)
              ], 
              verbose=True, 
              early_stopping_rounds = 10)
 
    # eval
    score = model.evals_result()['validation_1']['rmse'][-1]
    return score

In [0]:
# 学習結果を保存して、後から再利用できるようにしておく
study_name = 'xgbr_4'  # Unique identifier of the study.
study = optuna.create_study(pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),
                            study_name=study_name, 
                            storage='sqlite:///xgbr_4.db')

In [30]:
# 学習の実行
# https://optuna.readthedocs.io/en/stable/reference/study.html#optuna.study.Study.optimize
study.optimize(func=objective, # 実行する関数
               n_trials=20, # HPO試行回数
               timeout=None, # 与えられた秒数後に学習を中止します。default=None
               n_jobs=-1 # 並列実行するjob数
              )

[W 2019-05-10 14:02:26,244] Setting status of trial#115 as TrialState.FAIL because of the following error: TypeError("__call__() missing 1 required positional argument: 'inputs'",)
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/optuna/study.py", line 399, in _run_trial
    result = func(trial)
  File "<ipython-input-28-a5cd710b53dd>", line 39, in objective
    early_stopping_rounds = 10)
  File "/usr/local/lib/python3.6/dist-packages/keras/wrappers/scikit_learn.py", line 139, in fit
    **self.filter_sk_params(self.build_fn.__call__))
TypeError: __call__() missing 1 required positional argument: 'inputs'
[W 2019-05-10 14:02:28,922] Setting status of trial#114 as TrialState.FAIL because of the following error: TypeError("__call__() missing 1 required positional argument: 'inputs'",)
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/optuna/study.py", line 399, in _run_trial
    result = func(trial)
  File "<ipython-input-

In [0]:
'''
モデルの構築
'''
model = XGBRegressor(
        # selected
#         max_depth=3,
#         n_estimators=40,
        # def
        max_depth = 4,
        n_estimators = 60,
        min_child_weight=300, 
        colsample_bytree=0.8, 
        subsample=0.8, 
        eta=0.3, 
        n_jobs=-1,
        seed=42
    )

In [0]:
model.fit(X_train, 
              Y_train, 
              eval_metric="rmse", 
              eval_set=[
                  (X_train, Y_train),
                  (X_val, Y_val)
              ], 
              verbose=True, 
              early_stopping_rounds = 10)